## XGBoost-Optunaのサンプルコード

In [1]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# XGBoost
import xgboost as xgb

# Optuna
import optuna
from sklearn.model_selection import cross_val_score

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns = boston.feature_names)
# 目的変数の追加
df['MEDV'] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [3]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:, 0:df.shape[1]-1],
                                                    df.iloc[:, df.shape[1]-1],
                                                    test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE)

In [4]:
def objective(trial):
    
    eta =  trial.suggest_loguniform('eta', 1e-8, 1.0)
    gamma = trial.suggest_loguniform('gamma', 1e-8, 1.0)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_child_weight = trial.suggest_loguniform('min_child_weight', 1e-8, 1.0)
    subsample = trial.suggest_uniform('subsample', 0.0, 1.0)
    reg_lambda = trial.suggest_uniform('reg_lambda', 0.0, 1000.0)
    reg_alpha = trial.suggest_uniform('reg_alpha', 0.0, 1000.0)


    model = xgb.XGBRegressor(eta = eta,
                             gamma = gamma,
                             max_depth = max_depth,
                             min_child_weight = min_child_weight,
                             subsample = subsample,
                             reg_lambda = reg_lambda,
                             reg_alpha = reg_alpha
                            )

    score = cross_val_score(model,
                            x_train,
                            y_train,
                            cv=5,
                            scoring="neg_mean_absolute_error"
                           )
    print(score)
    mae = score.mean()

    return mae

In [5]:
%%time
# optunaで最適値を見つける
# 注：cross_val_scoreの出力は全て高いほど良い
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500)

[I 2021-05-08 12:06:29,760] A new study created in memory with name: no-name-22f0553f-2668-4f05-905f-a86b86a2be9f
[I 2021-05-08 12:06:31,178] Trial 0 finished with value: -21.0773022459911 and parameters: {'eta': 0.00013682736188883748, 'gamma': 0.19128850481175966, 'max_depth': 5, 'min_child_weight': 0.00010210519622976516, 'subsample': 0.8816389636614385, 'reg_lambda': 9.473689747664382, 'reg_alpha': 75.0819478304815}. Best is trial 0 with value: -21.0773022459911.


[-19.99185118 -21.94320454 -20.49658089 -21.059894   -21.89498062]


[I 2021-05-08 12:06:32,172] Trial 1 finished with value: -21.34375397662175 and parameters: {'eta': 2.3837521661873767e-07, 'gamma': 7.787403477034979e-06, 'max_depth': 13, 'min_child_weight': 5.8185143981883144e-08, 'subsample': 0.8123577889539625, 'reg_lambda': 775.5222031823389, 'reg_alpha': 795.6969417493282}. Best is trial 0 with value: -21.0773022459911.


[-20.25420935 -22.21594066 -20.75914828 -21.32458109 -22.16489051]


[I 2021-05-08 12:06:32,689] Trial 2 finished with value: -20.52326702679234 and parameters: {'eta': 0.0006502691747977047, 'gamma': 0.039348950475839015, 'max_depth': 11, 'min_child_weight': 0.0010160534059138109, 'subsample': 0.20860000065394202, 'reg_lambda': 7.480266958651338, 'reg_alpha': 450.3093579315707}. Best is trial 2 with value: -20.52326702679234.


...

[-2.36620123 -2.31156664 -2.3250797  -2.56300986 -3.02279042]


[I 2021-05-08 12:12:26,926] Trial 498 finished with value: -2.6755946073708707 and parameters: {'eta': 0.27649707641833954, 'gamma': 0.06406311960608245, 'max_depth': 6, 'min_child_weight': 0.00017639499220538666, 'subsample': 0.8084471381016022, 'reg_lambda': 82.54025318029711, 'reg_alpha': 48.87859178444825}. Best is trial 414 with value: -2.1627999439062897.


[-2.19476978 -2.55158366 -2.58711991 -2.60316209 -3.4413376 ]


[I 2021-05-08 12:12:28,391] Trial 499 finished with value: -2.4323250863581527 and parameters: {'eta': 0.21703309716044514, 'gamma': 0.07031181370580056, 'max_depth': 5, 'min_child_weight': 7.349184716136029e-05, 'subsample': 0.7914437865648652, 'reg_lambda': 37.53008022424496, 'reg_alpha': 18.6029128601035}. Best is trial 414 with value: -2.1627999439062897.


[-2.07395816 -2.23299532 -2.3687257  -2.41169112 -3.07425513]
CPU times: user 11min 9s, sys: 14.7 s, total: 11min 24s
Wall time: 5min 58s


In [6]:
# チューニングしたハイパーパラメーターをフィット
optimised_model = xgb.XGBRegressor(eta = study.best_params['eta'],
                                   gamma = study.best_params['gamma'],
                                   max_depth = study.best_params['max_depth'],
                                   min_child_weight = study.best_params['min_child_weight'],
                                   subsample = study.best_params['subsample'],
                                   reg_lambda = study.best_params['reg_lambda'],
                                   reg_alpha = study.best_params['reg_alpha']
                                  )

optimised_model.fit(x_train ,y_train)

# XGBoost推論
y_pred = optimised_model.predict(x_test)

In [7]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame({'R2': r2_score(true, pred),
                          'MAE': mean_absolute_error(true, pred),
                          'MSE': mean_squared_error(true, pred),
                          'RMSE': np.sqrt(mean_squared_error(true, pred))},
                           index = ['scores'])
    return scores

In [8]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE       MSE      RMSE
scores  0.808442  2.989264  20.03328  4.475855
